# Lab 3 - OpenAI Agents SDK + TOOLS!!

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
import gradio as gr
import os
import requests
load_dotenv(override=True)

In [3]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [4]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}

@function_tool
def get_ticket_price(destination_city: str) -> str:
    """ Get the ticket price from the database for a return trip to the given city

    Args:
        destination_city: The city for which the ticket price is requested

    """

    print(f"Tool get_ticket_price called for {destination_city}", flush=True)
    city = destination_city.lower()
    price = ticket_prices.get(city, "Unknown")
    return f"The price for a return trip to {destination_city} is {price}"

In [ ]:
get_ticket_price("London")

In [ ]:
get_ticket_price

In [ ]:
get_ticket_price.params_json_schema

In [8]:
@function_tool
def push(message: str):
    """Send a text message as a push notification to the user with this brief message

    Args:
        message: The short text message to push to the user
    """

    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [9]:
instructions = "You are a helpful assistant for an Airline called FlightAI. "
instructions += "Give short, witty, humorous answers, no more than 1 sentence. "
instructions += "Always be accurate. If you don't know the answer, say so."

In [10]:
agent = Agent(name="FlightAI", instructions=instructions, model="gpt-4.1-mini", tools=[get_ticket_price, push])


In [11]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = await Runner.run(agent, messages)
    return response.final_output

In [ ]:
gr.ChatInterface(chat, type="messages").launch()